# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

* Framework 위에서 Code를 짜고,Code에서 Library를 활용한다.<br>
즉 Framework는 뼈대가 되며 사용자를 제어하여 사용자는 Framework가 짜 놓은 틀 안에서 수동적으로 동작한다.<br>
반면 Library의 경우 사용자가 코드를 작성할 때 Library를 직접 제어하므로 그 제어 권한이 사용자에게 있다. 
---

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

* 관계<br>
머신러닝이 딥러닝을 포함한다.<br>
딥러닝은 머신러닝보다 작은 개념으로, '신경망'을 통해 인공지능을 만드는,<br> 즉 딥 뉴럴 네트워크 알고리즘을 이용하는 머신러닝의 한 종류이다.

* 특징<br>
머신러닝은 데이터를 학습하고 패턴을 분석하며, 학습이 진행되는 과정에서 오류를 줄여가며 가장 합리적인 모델을 찾아 예측 정확도를 높여가는 것을 목표로 한다.
<br>딥러닝은 머신러닝에 포함되는 개념으로,'분류를 통한 예측'을 핵심으로 작동한다.<br>
머신러닝은 사람이 직접 주요한 feature를 제공해주는 수동적인 학습을 진행하나, 딥러닝은 이를 자동적으로 골라내는 작업을 한다.

* 차이<br>
딥러닝은 분류에 사용될 데이터를 스스로 학습하나 머신러닝은 학습 데이터를 가공하거나 특정 feature에 대한 정보에 대한 정보만을 학습할 수 있다.<br><br>
머신러닝은 주어진 데이터를 인간이 먼저 처리한다. 즉 사람이 먼저 컴퓨터에 특정 패턴을 추출하는 방법을 지시하고, 이후 컴퓨터가 스스로 데이터의 특징을 분석하고 축적한다.
<br>반면 딥러닝은 컴퓨터가 스스로 데이터를 기반으로 학습할 수 있도록 정해진 신경망을 컴퓨터에게 주고 학습을 수행해 심층 신경망을 통해 스스로 분류가 가능하다.<br><br>
머신러닝이 딥러닝보다 전처리 과정이 복잡하며, 딥러닝은 전처리를 파라미터 조정으로 대체하는 경향이 있다.
---

### Q3. 아래의 코드에 주석 달기.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms #데이터 전처리 

# pytorch GPU 사용

# decive 변수 선언
# gpu 사용 가능시 학습 진행 기기를 cpu가 아닌 cuda로
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

# cpu 연산 random seed 고정
torch.manual_seed(45)

# multi gpu 연산 random seed 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
    
print(device + " is available")

### gpu사용이 불가능하여 cpu로 진행하였음

cpu is available


In [2]:
# 하이퍼 파라미터 (학습 시)

#학습률 - 모델 매개변수 조절 비율 ( 작을수록 학습 속도 느려짐 )
learning_rate = 0.001
#배치 크기 - 매개변수 갱신 전 신경망을 통해 전파된 데이터 샘플 수 
batch_size = 100
#클래스 수
num_classes = 10
#학습할 epoch 수 - 데이터셋 반복 횟수
epochs = 5

In [3]:
#torchvision.datasets의 MNIST 데이터셋 로드 
#(모든 데이터셋은 샘플과 정답을 각각 변경하기 위한 transform과 target_transform 인자 포함)

train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True, #train_set만
    download = True,
    transform = transfroms.Compose([ #데이터 이미지 tensor로 변환하는 함수
        transfroms.ToTensor()  
        # 이미지를 Pytorch tensors 타입으로 변형 (0-255 값을 0-1 사이 값으로)
        # 즉 (image,label)의 데이터를 floatTensor로 변환 후 이미지픽셀을 0.-1. 사이로 조정
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [4]:
# 데이터 로더 생성(데이터를 DataLoader의 인자로 전달)
# 데이터셋을 iterable로 감싸고, 자동화된 batch,sampling,shuffle 및 multiprocess data loading 지원


# torch.utils.data.Dataset는 샘플과 정답(label) 저장 
# torch.utils.data.DataLoader는 Dataset을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체(iterable)로 감싼다. 

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size) #batch_size 100
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# input size
examples = enumerate(train_set) # enumerate - 인덱스와 원소로 이루어진 튜플 생성
batch_idx, (example_data, example_targets) = next(examples) # next - 반복 가능 객체의 다음 요소 반환
example_data.shape

#torch.Size([1, 28, 28])출력 - input size 28*28

torch.Size([1, 28, 28])

In [5]:
# ConvNet(컨볼루션 신경망)

class ConvNet(nn.Module):
    
  def __init__(self): # layer 정의
        super(ConvNet, self).__init__() 
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #in_channels,out_channels,kernel_size->filter 10
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        # in_channels - 입력 채널 수 (흑백 이미지의 경우 1, RGB값을 가진 이미지의 경우 3)
        # out_channels - 출력 채널 수 
        # kernel_size - 커널 사이즈,int/tuple 가능
        
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) #랜덤하게 뉴런을 종료해서 학습을 방해해 overfitting 방지,p:존속 확률
        
        self.mp = nn.MaxPool2d(2) #pooling layer(kernel size)
        #max pooling:정해진 크기 안에서 가장 큰 값만 뽑아낸다 (텐서의 크기 축소,overfitting방지,특징 뽑아냄,일반화 성능)
        
        self.fc1 = nn.Linear(320,100) #input_size, output_size
        self.fc2 = nn.Linear(100,10) 

        
  def forward(self, x): #실제로 데이터가 거치는 부분
        x = F.relu(self.mp(self.conv1(x))) # convolution layer 1번에 relu를 씌우고 maxpool ,relu(x) = max(0,x)
        x = F.relu(self.mp(self.conv2(x))) # convolution layer 2번에 relu를 씌우고 maxpool
        x = self.drop2D(x) #드롭아웃
        x = x.view(x.size(0), -1) #텐서의 요소 개수는 유지한 채 형태만 변형 - Flatten 작업 
        x = self.fc1(x) # fc1 레이어에 삽입
        x = self.fc2(x) # fc2 레이어에 삽입
        return F.log_softmax(x) #활성함수 softmax에 log 취해 씌움

http://aikorea.org/cs231n/convolutional-networks/ 참고

In [6]:
# CNN instance 생성 ( 모델 선언)
model = ConvNet().to(device) 

# Cost Function, Optimizer 선택
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [7]:
#train

for epoch in range(epochs): # epochs 수만큼 반복
    avg_cost = 0
    
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() # gradient 값을 0으로 초기화 (미분한 값들이 누적되는 특징 때문)
        hypothesis = model(data) # forwawrd 연산
        cost = criterion(hypothesis, target) # 비용함수 - output과 target의 cost 계산
        cost.backward() # backward 함수를 호출해 gradient 계산
        optimizer.step() # 모델의 학습 파라미터 갱신
        avg_cost += cost / len(train_loader) # cost 값을 변수에 누적하고 train_loader 개수로 나눠 avg_cost
        
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

C:\Users\Kim Eun ho\AppData\Local\Temp\ipykernel_20116\1100380364.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) #활성함수 softmax에 log 취해 씌움


[Epoch:    1]  cost = 0.329661995
[Epoch:    2]  cost = 0.111260787
[Epoch:    3]  cost = 0.0849511698
[Epoch:    4]  cost = 0.0730437636
[Epoch:    5]  cost = 0.0651627332


In [8]:
#test

model.eval() # evaluate mode로 전환,dropout/batch_normalization 해제
with torch.no_grad(): # grad 해제 (gradient tracking) 하지 않음)
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] # 출력이 분류 각각에 대한 값으로 나타나므로 가장 높은 값을 갖는 인덱스 추출
        total += len(target) # 전체 클래스 개수
        correct += (preds==target).sum().item() # 예측값과 실제값 비교 -> 정확도 계산
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

C:\Users\Kim Eun ho\AppData\Local\Temp\ipykernel_20116\1100380364.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) #활성함수 softmax에 log 취해 씌움


Test Accuracy:  98.68 %
